In [19]:
import pandas as pd
import numpy as np
import pickle
from tqdm.auto import tqdm
from pprint import pprint
import os
import sys

sys.path.append('../')

In [20]:
data = pd.read_csv('../data/wiki_movies/filtered_wiki_movies_scraped.csv')

In [21]:
data.head()
data.drop_duplicates(subset=['Title'], inplace=True)
data.rename(columns={"Release Year": "Year", "Origin/Ethnicity": "Origin", "scraped_summary": "Plot Summary"}, inplace=True)
data.drop("Unnamed: 0", inplace=True, axis=1)

In [22]:
data.head(3)

,Year,Title,Origin,Director,Cast,Genre,Wiki Page,Plot,Plot Summary
0,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",biographical,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...,"Daniel Boone; or, Pioneer Days in America is a..."
1,1907,Laughing Gas,American,Edwin Stanton Porter,"Bertha Regustus, Edward Boulden",comedy,https://en.wikipedia.org/wiki/Laughing_Gas_(fi...,The plot is that of a black woman going to the...,Laughing Gas is the title of several American ...
2,1908,The Adventures of Dollie,American,D. W. Griffith,"Arthur V. Johnson, Linda Arvidson",drama,https://en.wikipedia.org/wiki/The_Adventures_o...,On a beautiful summer day a father and mother ...,The Adventures of Dollie is a 1908 American si...


In [23]:
data["Plot"] = data["Plot"].astype("string")
data["Plot Summary"] = data["Plot Summary"].astype("string")

def doc_length(text):
    return len(str(text))

data['Plot_len'] = data['Plot'].map(doc_length)
data['Plotsum_len'] = data['Plot Summary'].map(doc_length)
data['len_ratio'] = data['Plotsum_len']/data['Plot_len']

# print(data['Title'].unique().shape)

print(len(data))
print(len(data[data['Plotsum_len'] > 20]))
print(len(data[data['Plot_len'] > 100]))
print(len(data[data['len_ratio'] <= 0.5]))

data = data[data['Plotsum_len'] > 20]
data = data[data['Plot_len'] > 100]
data = data[data['len_ratio'] <= 0.5]
print(len(data))

26198
26162
25885
21078
21038


In [24]:
# data['Plot_len'].describe()

In [25]:
# count_series = data.groupby(['Title']).size().reset_index(name='count')['count']
# count_series[count_series>1]

# data.iloc[13916]

In [26]:
# with open('../data/wiki_movies/filtered_data_wiki_movies.pickle', 'wb') as f:
#     pickle.dump(data, f)

In [27]:
def parse_cols(df, parse_func_dict):
    for idx in tqdm(range(df.shape[0])):
        for k in parse_func_dict:
            print(parse_func_dict[k](df[k].iloc[idx]))

def parse_year(year):
    return str(year)

def parse_origin(origin):
    return str(origin)

def parse_director(director_str):
    directors = []
    director_str = director_str.lower()
    space_to = ""
    director_str = director_str.replace("\n", "")
    parse_by_comma = director_str.split(", ")
    for i, a in enumerate(parse_by_comma):
        if 'by' in a:
            a = a.split('by')[-1]
        # parse by 'and'
        if 'and' in a:
            al = []
            for v in a.split(' and '):
                tmp = v.replace(" ", space_to)
                if len(tmp)!=0: al.append(tmp)
            a = al[:]
                    
        if type(a) is list:
            directors.extend(a)
        else:
            if len(a.replace(" ", space_to))!=0:
                directors.append(a.replace(" ", space_to))
    return directors

def parse_genre(genre_str):
    genres = []
    genre_str = genre_str.lower()
    space_to = ""
    genre_str = genre_str.replace("\n", "")
    parse_by_comma = genre_str.split(", ")
    for i, a in enumerate(parse_by_comma):
        if 'by' in a:
            a = a.split('by')[-1]
        # parse by 'and'
        if 'and' in a:
            al = []
            for v in a.split(' and '):
                tmp = v.replace(" ", space_to)
                if len(tmp)!=0: al.append(tmp)
            a = al[:]
                    
        if type(a) is list:
            genres.extend(a)
        else:
            if len(a.replace(" ", space_to))!=0:
                genres.append(a.replace(" ", space_to))
    return genres

def parse_casts(cast_str):
    casts = []
    cast_str = cast_str.lower()
    space_to = ""
    cast_str = cast_str.replace("\n", "")
    parse_by_comma = cast_str.split(", ")
    for i, a in enumerate(parse_by_comma):
        if 'by' in a:
            a = a.split('by')[-1]
        # parse by 'and'
        if 'and' in a:
            al = []
            for v in a.split(' and '):
                tmp = v.replace(" ", space_to)
                if len(tmp)!=0: al.append(tmp)
            a = al[:]
                    
        if type(a) is list:
            casts.extend(a)
        else:
            if len(a.replace(" ", space_to))!=0:
                casts.append(a.replace(" ", space_to))
    return casts

parse_func_dict = {
    "Cast": parse_casts
}
parse_cols(data, parse_func_dict)


  0%|          | 0/21038 [00:00<?, ?it/s]

['williamcraven', 'florencelawrence']
['arthurv.johnson', 'lindaarvidson']
['florenceauer', 'johng.adolfi']
['arthurv.johnson']
['marypickford', 'henryb.walthall']
['charlesogle', 'nataliejerome']
['edgarg.wynn']
['elmerbooth', 'lilliangish']
['fordsterling', 'mabelnormand']
['williamgarwood', 'marieeline']
['louiseglaum']
['j.warrenkerrigan', 'paulinebush']
['maemarsh', 'clairemcdowell']
['henryb.walthall', 'blanchesweet']
['donaldcrisp', 'liliangish']
['edwardabeles', 'sydneydeane', 'josephsingleton']
['charliechaplin', 'mackswain', 'phyllisallen']
['charliechaplin', 'mackswain', 'alicedavenport']
['marypickford', 'owenmoore', 'isobelvernon']
['charliechaplin', 'edgarkennedy', 'mintadurfee']
['charliechaplin', 'chesterconklin', 'fritzschade']
['donaldcrisp', 'blanchesweet', 'maemarsh']
['charliechaplin', 'mabelnormand', 'phyllisallen']
['earlefoxe']
['earlefoxe', 'henryb.walthall', 'dorothygish']
['charliechaplin', 'mabelnormand']
['charliechaplin', 'mabelnormand']
['charliechaplin',

In [28]:
# load pickle
df = pd.read_pickle('../data/wiki_movies/filtered_data_wiki_movies.pickle')

In [29]:
i = 10

print(df.iloc[i]['Title'])
print(df.iloc[i]['Plot'])
print()
print(df.iloc[i]['Plot Summary'])

The Quakeress
The setting is an early American village, where a young Quaker woman, Priscilla (played by Glaum), is in love with the schoolmaster, John Hart (played by Ray). The local minister, Rev. Cole (played by Taylor), who calls on her at her cabin with flowers, is an unwelcome suitor. In revenge, he has "blue laws" passed, among them is one requiring attendance at church on Sunday.
Priscilla refuses to comply with the law and is arrested. After being plunged in and out of water and pilloried, she is banished from the colony. John goes with her. They are attacked by Indians and John is badly wounded. Priscilla manages to get back to the village in time to warn the Puritans of an impending attack. They defeat the Indians after a desperate battle.
The Rev. Cole, who has been mortally wounded, begs Priscilla's forgiveness and the Puritans make amends for their harsh treatment of her.[1]

The Quakeress is a 1913 silent era short costume drama motion picture starring Louise Glaum, Char

In [31]:
df[df['Title'] == 'Aakasamantha']

,Year,Title,Origin,Director,Cast,Genre,Wiki Page,Plot,Plot Summary,Plot_len,Plotsum_len,len_ratio
26656,2009,Aakasamantha,Telugu,Radha Mohan,"Prakash Raj, Trisha Krishnan, Jagapati Babu",drama,https://en.wikipedia.org/wiki/Aakasamantha,During a walk to the park Raghuraman (Prakash ...,Abhiyum Naanum (transl. Abhi and I) is a 2008 ...,1889,510,0.269984


In [32]:
wiki_graph = pd.read_pickle('../data/wiki_movies/graph.pickle')

In [33]:
dk = list(wiki_graph.data_dict.keys())

for c in dk:
    # if "Aakasamantha" in c:
    if "Aak" in c:
        print(c)

Aakasamantha
Aakhri Adaalat
Aakrosh
Aakhir Kyon?


In [10]:
dk

['Perfect Stranger',
 'Udan Khatola',
 'Needful Things',
 'Chicken and Duck Talk',
 'Garam Hawa',
 'The Darkness',
 'The Last Detail',
 'Dishyum',
 'Woman on Top',
 'Goon: Last of the Enforcers',
 'Bandit Queen',
 'Goodbye Mr. Loser',
 'Sauda',
 'Saradaga Ammayitho',
 'Valiant',
 'The Missing Corpse',
 'Riffraff',
 'Full Metal Jacket',
 'Disciples of the 36th Chamber',
 'Top Gun',
 'Gentlemen Are Born',
 'Friday the 13th Part 2',
 'Indra',
 ' Strangerland',
 'Hands Across the Table',
 'Flight from Glory',
 'All I Desire',
 'Haunted – 3D',
 ' Joshua',
 'Junior',
 'A Zed & Two Noughts',
 'Geraftaar',
 'Naruto Shippūden 5: Blood Prison',
 'Zombie Hunter',
 'Win a Date with Tad Hamilton!',
 'Pullipulikalum Aattinkuttiyum',
 'Drum Beat',
 'Paramanandayya Shishyula Katha',
 'Nobody Walks',
 'Dulha Mil Gaya',
 'Cheater',
 'The Mouse on the Moon',
 'Be With You',
 'The Landlord',
 'Youngblood Hawke',
 'English Without Tears',
 'Salute John Citizen',
 'Hallam Foe',
 'Equinox Flower',
 'The Lege